# Notebook 6: Rayleigh-Bénard Convection (time-stepping example)

<div style="float: right; width: 40%">
    
![](media/AnnulusConvectionModel.png)

</div>


We'll look at a convection problem which couples Stokes Flow with time-dependent advection/diffusion to give simple Rayleigh-Bénard convection model. 

$$
-\nabla \cdot
    \left[
            \frac{\eta}{2}\left( \nabla \mathbf{u} + \nabla \mathbf{u}^T \right) -  p \mathbf{I} \right] =
     -\rho_0 \alpha T \mathbf{g} 
$$
$$
\nabla \cdot \mathbf{u} = 0
$$

$\eta$ is viscosity, $p$ is pressure, $\rho_0$ is a reference density, $\alpha$ is thermal expansivity, and $T$ is the temperature. Here we explicitly express density variations in terms of temperature variations.

Thermal evolution is given by
$$
\frac{\partial T}{\partial t} - \mathbf{u}\cdot\nabla T = \kappa \nabla^2 T 
$$
where the velocity, $\mathbf{u}$ is the result of the Stokes flow calculation. $\kappa$ is the thermal diffusivity (compare this with Notebook 4).  

The starting point is our previous notebook where we solved for Stokes
flow in a cylindrical annulus geometry. We then add an advection-diffusion 
solver to evolve temperature. The Stokes buoyancy force is proportional to the
temperature anomaly, and the velocity solution is fed back into the 
temperature advection term. The timestepping loop is written by
hand because usually you will want to do some analysis or output some checkpoints.

To read more about the applications of simple mantle convection models like this one, see (for example) Schubert et al, 2001.



In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio

nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import numpy as np
import sympy
import underworld3 as uw

In [3]:
res = 10
r_o = 1.0
r_i = 0.55

rayleigh_number = 3.0e4

meshball = uw.meshing.Annulus(
    radiusOuter=r_o,
    radiusInner=r_i,
    cellSize=1 / res,
    qdegree=3,
)

# Coordinate directions etc
x, y = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.xR
unit_rvec = meshball.CoordinateSystem.unit_e_0

# Orientation of surface normals
Gamma_N = unit_rvec

In [4]:
# Mesh variables for the unknowns

v_soln = uw.discretisation.MeshVariable("V0", meshball, 2, degree=2, varsymbol=r"{v_0}")
p_soln = uw.discretisation.MeshVariable("p", meshball, 1, degree=1, continuous=True)
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3, continuous=True)

### Create linked solvers

We create the Stokes solver as we did in the previous notebook. 
The buoyancy force is proportional to the temperature anomaly
(`t_soln`). Solvers can either be provided with unknowns as pre-defined
meshVariables, or they will define their own. When solvers are coupled,
explicitly defining unknowns makes everything clearer.

The advection-diffusion solver evolved `t_soln` using the Stokes
velocity `v_soln` in the fluid-transport term. 

### Curved, free-slip boundaries

In the annulus, a free slip boundary corresponds to zero radial 
velocity. However, in this mesh, $v_r$ is not one of the unknowns
($\mathbf{v} = (v_x, v_y)$). We apply a non linear boundary condition that
penalises $v_r$ on the boundary as discussed previously in Example 5. 

In [5]:
stokes = uw.systems.Stokes(
    meshball,
    velocityField=v_soln,
    pressureField=p_soln,
)

stokes.bodyforce = rayleigh_number * t_soln.sym * unit_rvec

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.shear_viscosity_0 = 1
stokes.tolerance = 1.0e-3

stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"

stokes.add_natural_bc(1000000 * Gamma_N.dot(v_soln.sym) * Gamma_N, "Upper")

if r_i != 0.0:
    stokes.add_natural_bc(1000000 * Gamma_N.dot(v_soln.sym) * Gamma_N, "Lower")

In [7]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_fn=v_soln,
    order=2,
    verbose=False,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = 1

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(+1.0, "Lower")
adv_diff.add_dirichlet_bc(-0.0, "Upper")

adv_diff.petsc_options.setValue("snes_rtol", 0.001)
adv_diff.petsc_options.setValue("ksp_rtol", 0.0001)
adv_diff.petsc_options.setValue("snes_monitor", None)
# adv_diff.petsc_options.setValue("ksp_monitor", None)

In [8]:
uw.systems.AdvDiffusion.view()


# Advection-Diffusion Equation Solver (Scalar Variable)

This class provides a solver for the scalar Advection-Diffusion equation using the characteristics based Semi-Lagrange Crank-Nicholson method
which is described in Spiegelman & Katz, (2006).

$$
\color{Green}{\underbrace{ \Bigl[ \frac{\partial u}{\partial t} + \left( \mathbf{v} \cdot \nabla \right) u \Bigr]}_{\dot{\mathbf{u}}}} -
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[ \boldsymbol\kappa \nabla u \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ f \Bigl] }_{\mathbf{h}}}
$$

The term $\mathbf{F}$ relates diffusive fluxes to gradients in the unknown $u$. The advective flux that results from having gradients along
the direction of transport (given by the velocity vector field $\mathbf{v}$ ) are included in the $\dot{\mathbf{u}}$ term.

The term $\dot{\mathbf{u}}$ involves upstream sampling to find the value $u^*$ which represents the value of $u$ at
the points which later arrive at the nodal points of the mesh. This is achieved using a "hidden"
swarm variable which is advected backwards from the nodal points automatically during the `solve` phase.

## Properties

  - The unknown is $u$.

  - The velocity field is $\mathbf{v}$ and is provided as a `sympy` function to allow operations such as time-averaging to be
    calculated in situ (e.g. `V_Field = v_solution.sym`) **NOTE: no it's not.

  - The diffusivity tensor, $\kappa$ is provided by setting the `constitutive_model` property to
    one of the scalar `uw.constitutive_models` classes and populating the parameters.
    It is usually a constant or a function of position / time and may also be non-linear
    or anisotropic.

  - Volumetric sources of $u$ are specified using the $f$ property and can be any valid combination of `sympy` functions of position and
    `meshVariable` or `swarmVariable` types.

## References

Spiegelman, M., & Katz, R. F. (2006). A semi-Lagrangian Crank-Nicolson algorithm for the numerical solution of advection-diffusion problems. Geochemistry, Geophysics, Geosystems, 7(4). https://doi.org/10.1029/2005GC001073



In [9]:
uw.systems.SemiLagragian_DDt.view()


# Nodal-Swarm  Semi-Lagrangian History Manager:

This manages the semi-Lagrangian update of a Mesh Variable, $\psi$, on the mesh across timesteps.
$$\quad \psi_p^{t-n\Delta t} \leftarrow \psi_p^{t-(n-1)\Delta t}\quad$$
$$\quad \psi_p^{t-(n-1)\Delta t} \leftarrow \psi_p^{t-(n-2)\Delta t} \cdots\quad$$
$$\quad \psi_p^{t-\Delta t} \leftarrow \psi_p^{t}$$


#### Underworld expressions

Note that the parameters in the consititutive models are defined as `uw.expression`s because 
  1. We generally would like to display them symbolically in equations rather than see their floating point value
  2. They can represent complex expressions that "blow up" the symbolic forms when we inspect the equations
  3. We would like to be able to substitute new values in the equations and use the *lazy evaluation* of expressions to avoid having to redefine expressions in multiple locations where these symbols appear
  4. For future reference, in optimisation problems, we differentiate expressions with respect to their parameters and want the results also to be evaluated in a lazy fashion.

In [10]:
display(type(stokes.constitutive_model.Parameters.shear_viscosity_0))
display(type(adv_diff.constitutive_model.Parameters.diffusivity))

stokes.constitutive_model.Parameters.shear_viscosity_0.sym

underworld3.function.expressions.UWexpression

underworld3.function.expressions.UWexpression

1

### Initial condition

We need to set an initial condition for the temperature field as the 
coupled system is an initial value problem. Choose whatever works but
remember that the boundary conditions will over-rule values you set on 
the lower and upper boundaries.

In [11]:
# Initial temperature

init_t = 0.1 * sympy.sin(3 * th) * sympy.cos(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)

t_soln.array[...] = uw.function.evaluate(init_t, t_soln.coords)


#### Initial velocity solve

The first solve allows us to determine the magnitude of the velocity field 
and is useful to keep separated to check convergence rates etc. 

For non-linear problems, we usually need an initial guess using a 
reasonably close linear problem. 

`zero_init_guess` is used to reset any information in the vector of 
unknowns (i.e. do not use any initial information if `zero_init_guess==True`).

In [12]:
stokes.solve(zero_init_guess=True)

In [13]:
# Keep the initialisation separate
# so we can run the loop again in a notebook

max_steps = 25
timestep = 0
elapsed_time = 0.0

In [14]:
adv_diff.view(class_documentation=True)


# Advection-Diffusion Equation Solver (Scalar Variable)

This class provides a solver for the scalar Advection-Diffusion equation using the characteristics based Semi-Lagrange Crank-Nicholson method
which is described in Spiegelman & Katz, (2006).

$$
\color{Green}{\underbrace{ \Bigl[ \frac{\partial u}{\partial t} + \left( \mathbf{v} \cdot \nabla \right) u \Bigr]}_{\dot{\mathbf{u}}}} -
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[ \boldsymbol\kappa \nabla u \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ f \Bigl] }_{\mathbf{h}}}
$$

The term $\mathbf{F}$ relates diffusive fluxes to gradients in the unknown $u$. The advective flux that results from having gradients along
the direction of transport (given by the velocity vector field $\mathbf{v}$ ) are included in the $\dot{\mathbf{u}}$ term.

The term $\dot{\mathbf{u}}$ involves upstream sampling to find the value $u^*$ which represents the value of $u$ at
the points which later arrive at the nodal points of the mesh. This is achieved using a "hidden"
swarm variable which is advected backwards from the nodal points automatically during the `solve` phase.

## Properties

  - The unknown is $u$.

  - The velocity field is $\mathbf{v}$ and is provided as a `sympy` function to allow operations such as time-averaging to be
    calculated in situ (e.g. `V_Field = v_solution.sym`) **NOTE: no it's not.

  - The diffusivity tensor, $\kappa$ is provided by setting the `constitutive_model` property to
    one of the scalar `uw.constitutive_models` classes and populating the parameters.
    It is usually a constant or a function of position / time and may also be non-linear
    or anisotropic.

  - Volumetric sources of $u$ are specified using the $f$ property and can be any valid combination of `sympy` functions of position and
    `meshVariable` or `swarmVariable` types.

## References

Spiegelman, M., & Katz, R. F. (2006). A semi-Lagrangian Crank-Nicolson algorithm for the numerical solution of advection-diffusion problems. Geochemistry, Geophysics, Geosystems, 7(4). https://doi.org/10.1029/2005GC001073



---

**Class**: <class 'underworld3.systems.solvers.SNES_AdvectionDiffusion'>

# Underworld / PETSc General Scalar Equation Solver

Primary problem: 

...

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Lower | $\left[\begin{matrix}1.0\end{matrix}\right]  $ | 
| **essential** | Upper | $\left[\begin{matrix}0.0\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [15]:
init_t = 0.1 * sympy.sin(3 * th) * sympy.cos(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)


t_soln.array[...] = uw.function.evaluate(
        init_t + 0.0001 * t_soln.sym[0], adv_diff.DuDt.psi_star[0].coords
    )

In [16]:
adv_diff.estimate_dt().squeeze()

array(0.00153108)

In [17]:
adv_diff.solve(timestep=0.01, zero_init_guess=True)

  0 SNES Function norm 9.480075146843e+00
  1 SNES Function norm 3.379297801722e-03


In [18]:
# Null space ?

for step in range(0, max_steps):

    stokes.solve(zero_init_guess=False)
    delta_t = 2 * adv_diff.estimate_dt()
    adv_diff.solve(timestep=delta_t, zero_init_guess=False, verbose=False)

    timestep += 1
    elapsed_time += delta_t

    if timestep % 5 == 0:
        print(f"Timestep: {timestep}, time {elapsed_time}")

  0 SNES Function norm 2.537329885868e-01
  1 SNES Function norm 3.399225908717e-05
  0 SNES Function norm 1.859196730018e-01
  1 SNES Function norm 2.970445884435e-05
  0 SNES Function norm 1.983538468982e-01
  1 SNES Function norm 2.624851485840e-05
  0 SNES Function norm 2.366852129209e-01
  1 SNES Function norm 2.609257586118e-05
  0 SNES Function norm 2.940972835405e-01
  1 SNES Function norm 2.941130851455e-05
Timestep: 5, time 0.015310756944410532
  0 SNES Function norm 3.731014256699e-01
  1 SNES Function norm 4.061905819818e-05
  0 SNES Function norm 4.773780460313e-01
  1 SNES Function norm 6.134672191790e-05
  0 SNES Function norm 6.237032139054e-01
  1 SNES Function norm 6.678168696271e-05
  0 SNES Function norm 7.999637427320e-01
  1 SNES Function norm 6.685247540654e-05
  0 SNES Function norm 9.544737343060e-01
  1 SNES Function norm 1.341436023111e-04
Timestep: 10, time 0.02720504969596519
  0 SNES Function norm 1.111253211674e+00
  1 SNES Function norm 1.611616435719e-0

In [19]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)

    skip = 1
    points = np.zeros((meshball._centroids[::skip].shape[0], 3))
    points[:, 0] = meshball._centroids[::skip, 0]
    points[:, 1] = meshball._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        integrator_type=45,
        surface_streamlines=True,
        initial_step_length=0.01,
        max_time=1.0,
        max_steps=500,
    )

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False,
    )

    pl.add_mesh(
        pvstream,
        opacity=0.33,
        show_scalar_bar=False,
        cmap="Greens",
        render_lines_as_tubes=False,
        
    )

    pl.export_html("html5/annulus_convection_plot.html")
    # pl.show(cpos="xy", jupyter_backend="trame")

In [20]:
#| fig-cap: "Interactive Image: Convection model output"
from IPython.display import IFrame

IFrame(src="html5/annulus_convection_plot.html", width=500, height=400)

## Exercise - Null space

Based on our previous notebook, can you see how to calculate and (if necessary) remove rigid-body the rotation 
null-space from the solution ? 

The use of a coarse-level singular-value decomposition for the velocity solver should help, in this case, but sometimes
you can see that there is a rigid body rotation (look at the streamlines). It's wise to check and quantify the presence of 
the null space.

```python
    stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
```

## Exercise - Heat flux

Could you calculate the radial heat flux field ? Its surface average value plotted against
time tells you if you have reached a steady state.

Hint:

$$
    Q_\textrm{surf} = \nabla T \cdot \hat{r} + T (\mathbf{v} \cdot \hat{r} )
$$ 

```python
    Q_surf = -meshball.vector.gradient(t_soln.sym).dot(unit_rvec) +\
                    t_soln.sym[0] * v_soln.sym.dot(unit_rvec)
```




### References

Schubert, G., Turcotte, D. L., & Olson, P. (2001). Mantle Convection in the Earth and Planets (1st ed.). Cambridge University Press. https://doi.org/10.1017/CBO9780511612879
